### Create documentation

**PROMPT**  
You are a subject matter expert on credit risk policies, regulatory compliance (e.g., Basel III),   
and internal governance standards. I have a set of documents covering regulations, internal   
guidelines, and reference materials. I also have a Table of Contents (ToC) that outlines how I   
want the final policy to be structured. 

Using only the content retrieved from the provided documents and guided by the ToC, please   
create each policy section. For each section, incorporate references to relevant regulatory   
requirements, define any necessary terminology, and outline best practices for credit risk   
assessment, approval workflows, monitoring, reporting, governance, and compliance. 

Keep the language concise, clear, and aligned with standard industry formats. Structure your   
final output according to the headings in the ToC, ensuring a coherent, well-organized policy   
that meets both regulatory and internal standards.

#### CreditRiskPolicyAgent Overview

- **Purpose**: Automate creation and refinement of a Credit Risk Policy document  
- **Components**:  
  - **DocumentCreator**: Fetches context from Chroma and drafts each section  
  - **DocumentEvaluator**: Reviews each draft, providing feedback for improvement  
  - **FinalReviewer**: Conducts a final holistic assessment of the entire policy  

#### Workflow

1. **Draft**:  
   The creator pulls relevant text from the Chroma database.  
   It then produces an initial section draft.  

2. **Evaluate**:  
   The evaluator checks accuracy, clarity, and compliance.  
   Feedback is generated for each section.  

3. **Refine**:  
   The creator refines the draft, based on evaluator feedback.  

4. **Final Review**:  
   Once all sections are ready, the FinalReviewer inspects the entire policy.  
   It ensures coherence and makes any final improvements.  

#### Key Methods

- **draft_section(section_title)**  
  Creates a preliminary version of a policy section.  

- **evaluate_section(draft_text)**  
  Reviews the text for correctness and suggests enhancements.  

- **refine_with_feedback(draft_text, feedback)**  
  Applies feedback to produce a better draft.  

- **review_document(entire_document)**  
  Performs the last check on the policy as a whole.  

#### Usage

1. **Initialize** the CreditRiskPolicyAgent with your API key and Chroma collection.  
2. **Provide** a table of contents for the policy.  
3. **Call** `build_policy`, which executes draft, evaluate, refine, and final review steps.  
4. **Receive** a fully formed policy document, ready for adoption.

In [8]:
import os
import logging

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain.callbacks import get_openai_callback

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DocumentCreator:
    """
    Creates and refines policy sections.
    Pulls context from Chroma by passing heading, overall doc, and section desc as the query.
    """
    def __init__(self, llm, retriever):
        logging.info("Initializing DocumentCreator...")
        self.llm = llm
        self.retriever = retriever
        logging.info("DocumentCreator initialized.")

    def retrieve_context(self, overall_desc, section_title, section_desc):
        logging.info(f"Retrieving context for section: '{section_title}'...")
        query_text = (
            f"Overall Document: {overall_desc}\n"
            f"Section Title: {section_title}\n"
            f"Section Description: {section_desc}"
        )
        # Retrieve relevant documents
        docs = self.retriever.get_relevant_documents(query_text)
        context = "\n".join([d.page_content for d in docs])
        logging.info(f"Context retrieved for section: '{section_title}'. Context length: {len(context)} characters.")
        return context

    def draft_section(self, overall_desc, section_title, section_desc):
        logging.info(f"Drafting section: '{section_title}'...")
        # Create a chat prompt with system and user instructions
        system_template = (
            "You are drafting a Credit Risk Policy section. "
            "Incorporate best practices and references."
        )
        user_template = (
            "Section Title: {section_title}\n"
            "Overall Description: {overall_desc}\n"
            "Section Description: {section_desc}\n"
            "Relevant Context:\n{context}\n\n"
            "Draft the policy section."
        )

        system_msg = SystemMessagePromptTemplate.from_template(system_template)
        human_msg = HumanMessagePromptTemplate.from_template(user_template)

        # Retrieve any relevant context from Chroma
        context = self.retrieve_context(overall_desc, section_title, section_desc)

        chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
        formatted_prompt = chat_prompt.format_messages(
            section_title=section_title,
            overall_desc=overall_desc,
            section_desc=section_desc,
            context=context
        )

        with get_openai_callback() as cb:
            response = self.llm.invoke(formatted_prompt)
            logging.info(f"LLM call for drafting section '{section_title}' completed.")
            logging.info(f"Drafting section '{section_title}' token usage: {cb.total_tokens} (Prompt: {cb.prompt_tokens}, Completion: {cb.completion_tokens}, Cost: ${cb.total_cost:.4f})")

        logging.info(f"Section '{section_title}' drafted.")
        return response.content

    def refine_with_feedback(self, draft, feedback):
        logging.info(f"Refining section with feedback...")
        system_template = (
            "You are refining a policy section draft based on reviewer feedback."
        )
        user_template = (
            "Original Draft:\n{draft}\n\n"
            "Reviewer Feedback:\n{feedback}\n\n"
            "Refine the draft, addressing all feedback."
        )

        system_msg = SystemMessagePromptTemplate.from_template(system_template)
        human_msg = HumanMessagePromptTemplate.from_template(user_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
        formatted_prompt = chat_prompt.format_messages(draft=draft, feedback=feedback)

        with get_openai_callback() as cb:
            response = self.llm.invoke(formatted_prompt)
            logging.info(f"LLM call for refining section with feedback completed.")
            logging.info(f"Refinement token usage: {cb.total_tokens} (Prompt: {cb.prompt_tokens}, Completion: {cb.completion_tokens}, Cost: ${cb.total_cost:.4f})")

        logging.info(f"Section refined with feedback.")
        return response.content

class DocumentEvaluator:
    """
    Evaluates individual sections for correctness, clarity, and compliance.
    Provides feedback for improvement.
    """
    def __init__(self, llm):
        logging.info("Initializing DocumentEvaluator...")
        self.llm = llm
        logging.info("DocumentEvaluator initialized.")

    def evaluate_section(self, draft_section):
        logging.info("Evaluating section...")
        system_template = "You are an independent reviewer of a policy draft."
        user_template = (
            "Draft Section:\n{draft_section}\n\n"
            "1) Evaluate correctness, clarity, and compliance.\n"
            "2) Suggest improvements.\n"
            "3) Provide concise feedback."
        )

        system_msg = SystemMessagePromptTemplate.from_template(system_template)
        human_msg = HumanMessagePromptTemplate.from_template(user_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
        formatted_prompt = chat_prompt.format_messages(draft_section=draft_section)

        with get_openai_callback() as cb:
            response = self.llm.invoke(formatted_prompt)
            logging.info("LLM call for section evaluation completed.")
            logging.info(f"Evaluation token usage: {cb.total_tokens} (Prompt: {cb.prompt_tokens}, Completion: {cb.completion_tokens}, Cost: ${cb.total_cost:.4f})")

        logging.info("Section evaluated, feedback generated.")
        return response.content

class FinalReviewer:
    """
    Conducts a holistic review of the entire assembled policy.
    Ensures coherence, consistency, and completeness.
    """
    def __init__(self, llm):
        logging.info("Initializing FinalReviewer...")
        self.llm = llm
        logging.info("FinalReviewer initialized.")

    def review_document(self, full_document):
        logging.info("Conducting final document review...")
        system_template = (
            "You are a senior reviewer conducting a final review "
            "of the entire policy."
        )
        user_template = (
            "Below is the entire policy:\n\n"
            "{full_document}\n\n"
            "1) Check consistency and completeness.\n"
            "2) Suggest improvements.\n"
            "3) Provide the final revised text."
        )

        system_msg = SystemMessagePromptTemplate.from_template(system_template)
        human_msg = HumanMessagePromptTemplate.from_template(user_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
        formatted_prompt = chat_prompt.format_messages(full_document=full_document)

        with get_openai_callback() as cb:
            response = self.llm.invoke(formatted_prompt)
            logging.info("LLM call for final document review completed.")
            logging.info(f"Final review token usage: {cb.total_tokens} (Prompt: {cb.prompt_tokens}, Completion: {cb.completion_tokens}, Cost: ${cb.total_cost:.4f})")

        logging.info("Final document review completed.")
        return response.content

class CreditRiskPolicyAgent:
    """
    Orchestrates creation, section-level evaluation, refinement,
    and a final holistic review.
    """
    def __init__(self, api_key, chroma_collection):
        logging.info("Initializing CreditRiskPolicyAgent...")
        embeddings = OpenAIEmbeddings(api_key=api_key)
        store = Chroma(collection_name=chroma_collection, embedding_function=embeddings)

        # ChatOpenAI is the newer recommended approach for OpenAI chat models
        self.llm = ChatOpenAI(model_name="gpt-4", api_key=api_key)

        self.creator = DocumentCreator(self.llm, store.as_retriever())
        self.evaluator = DocumentEvaluator(self.llm)
        self.final_reviewer = FinalReviewer(self.llm)
        logging.info("CreditRiskPolicyAgent initialized.")

    def build_policy(self, overall_desc, toc_dict):
        logging.info("Starting policy building process...")
        refined_sections = {}
        total_tokens_policy = 0
        total_cost_policy = 0

        for key, s_info in toc_dict.items():
            s_title = s_info["title"]
            s_desc = s_info["description"]
            logging.info(f"Processing section: '{s_title}'...")

            # 1) Draft
            draft = self.creator.draft_section(overall_desc, s_title, s_desc)
            # 2) Evaluate
            feedback = self.evaluator.evaluate_section(draft)
            # 3) Refine
            refined = self.creator.refine_with_feedback(draft, feedback)
            refined_sections[s_title] = refined
            logging.info(f"Section '{s_title}' processing complete.")

        # Combine all sections for final review
        entire_doc = ""
        for s_title, text in refined_sections.items():
            entire_doc += f"{s_title}\n\n{text}\n\n"

        final_policy_output = self.final_reviewer.review_document(entire_doc)

        final_policy = final_policy_output
        logging.info("Policy building process completed.")
        return final_policy

def main():
    api_key = os.environ["OPENAI_API_KEY"]
    chroma_collection = "risk_universe"

    overall_doc_description = (
        "This document sets out the company's Credit Risk Policy, "
        "aligned with Basel III and internal guidelines."
    )

    toc = {
        "section_1": {
            "title": "Introduction and Scope",
            "description": "High-level overview, disclaimers, and scope"
        },
        "section_2": {
            "title": "Definitions and Terminology",
            "description": "Define all key terms"
        },
        "section_3": {
            "title": "Risk Assessment Methodology",
            "description": "Methods for identifying, measuring, scoring risks"
        },
        "section_4": {
            "title": "Credit Approval Workflow",
            "description": "Steps, roles, responsibilities for approval"
        },
        "section_5": {
            "title": "Monitoring and Reporting",
            "description": "Ongoing monitoring frameworks and reporting"
        },
        "section_6": {
            "title": "Governance and Compliance",
            "description": "Governance structure and compliance needs"
        }
    }

    agent = CreditRiskPolicyAgent(api_key, chroma_collection)
    logging.info("Starting to build final document...")
    final_document = agent.build_policy(overall_doc_description, toc)
    logging.info("Final document build complete.")
    print("\nFINAL POLICY DOCUMENT:\n")
    print(final_document)

main()

2025-01-24 06:26:05,332 - INFO - Initializing CreditRiskPolicyAgent...
2025-01-24 06:26:07,061 - INFO - Initializing DocumentCreator...
2025-01-24 06:26:07,062 - INFO - DocumentCreator initialized.
2025-01-24 06:26:07,062 - INFO - Initializing DocumentEvaluator...
2025-01-24 06:26:07,063 - INFO - DocumentEvaluator initialized.
2025-01-24 06:26:07,063 - INFO - Initializing FinalReviewer...
2025-01-24 06:26:07,064 - INFO - FinalReviewer initialized.
2025-01-24 06:26:07,064 - INFO - CreditRiskPolicyAgent initialized.
2025-01-24 06:26:07,065 - INFO - Starting to build final document...
2025-01-24 06:26:07,066 - INFO - Starting policy building process...
2025-01-24 06:26:07,066 - INFO - Processing section: 'Introduction and Scope'...
2025-01-24 06:26:07,066 - INFO - Drafting section: 'Introduction and Scope'...
2025-01-24 06:26:07,067 - INFO - Retrieving context for section: 'Introduction and Scope'...
2025-01-24 06:26:07,361 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings 


FINAL POLICY DOCUMENT:

After a thorough review, the policy appears consistent and complete. It covers all the essential points needed for a robust credit risk policy, including the introduction and scope, best practices for credit risk management, regulatory compliance, policy review, exceptions, violations, definitions, risk assessment methodology, credit approval workflow, monitoring and reporting, and governance and compliance. 

The policy is also aligned with Basel III standards and regulatory requirements, which is crucial for any financial institution. The roles and responsibilities of different entities, such as the Board of Directors, Risk Management Committee, and Credit Department, are clearly defined. 

One suggestion for improvement could be to add a section on the consequences of policy violations. While Section 6 mentions that violations could lead to disciplinary action, it could be helpful to provide more details on the specific consequences, such as warnings, retrai